## Import Required Files

In [7]:
import pandas as pd
import numpy as np
import os

In [9]:
## change it to the unzip path of the downloaded dataset..
data_folder = r'/Users/anurag/Downloads/movie genre classification/Scripts'

In [10]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

Total Number of Files : 2827


# Read Train Files

In [11]:
train_df = pd.read_csv('Train.csv')
train_df

,Genre,File_Name
0,Fantasy,file_2180.txt
1,Comedy,file_693.txt
2,Drama,file_2469.txt
3,Drama,file_2542.txt
4,Sci-Fi,file_378.txt
...,...,...
1973,Thriller,file_1930.txt
1974,Thriller,file_1821.txt
1975,Sci-Fi,file_350.txt
1976,Thriller,file_1933.txt


In [15]:
test_df = pd.read_csv('Our_test.csv')
test_df

,Genre,File_Name
0,Drama,file_2300.txt
1,Comedy,file_809.txt
2,Action,file_1383.txt
3,Adventure,file_983.txt
4,Thriller,file_1713.txt
...,...,...
844,Drama,file_2474.txt
845,Comedy,file_863.txt
846,War,file_1547.txt
847,Action,file_1292.txt


In [27]:
## let's read the text scripts in the train and test dataframes..

train_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

# Lets look at a script file after Reading..

In [28]:
#lets check one of the scripts..
print(train_df['Script'][4][:3000])

<b><!--

</b>if (window!= top)

top.location.href=location.href

<b>// -->

</b>

The Abyss - by James Cameron 

                                  THE ABYSS

                            AN ORIGINAL SCREENPLAY

                                      BY

                                JAMES CAMERON

                               August 2, 1988

                             Director's Revision

------------------------------------------------------------------------------

                                  THE ABYSS

OMITTED                                                                 1

OMITTED                                                                 2

TITLE: THE ABYSS -- ON BLACK, DISSOLVING TO COBALT BLUE

EXT. OCEAN/UNDERWATER -- DAY                                            3

Blue, deep and featureless, the twilight of five hundred feet down.

PROPELLER SOUND.  Materializing out of the blue limbo is the enormous but

sleek form of an Ohio-class SSBN ballistic missile su

In [33]:
print(test_df['Script'][4][:3000])

                                          KING KONG

                                          Written by

                        Fran Walsh, Philippa Boyens and Peter Jackson

                                     Based on a Story by

                             Merian C. Cooper and Edgar Wallace

                                                                   1.

          EXT. CENTRAL PARK - DAY

          CLOSE ON: A scrawny MONKEY scratches.

          ANGLES ON: Defeated, listless ANIMALS, in the bleak environs of a

          dilapidated ZOO.

          WIDER: It is CENTRAL PARK ZOO in depression era NEW YORK. The PARK

          itself is like a GARBAGE DUMP, dotted with squalid SHANTY TOWNS.

          Against these BLEAK IMAGES, the SOUND of a BRIGHT, BRASSY SONG

          fades up: Al Jolson, singing "I'm Sitting on Top of the World".

          The sky line of MANHATTAN rises in the background, a grim steaming

          jungle on this cold FALL day.

                 

In [34]:
train_df.Genre.value_counts()

Drama        405
Thriller     261
Comedy       243
Action       203
Crime        141
Romance      134
Adventure    116
Sci-Fi       109
Horror       104
Fantasy       79
Mystery       75
Family        27
Animation     25
War           18
Musical       15
Western        9
Music          4
Film-Noir      3
Short          2
Biography      2
History        2
Sport          1
Name: Genre, dtype: int64

### As we notice that `genre == 'Sport'` has only 1 occurrence, Hence we can't stratify the split for validation data, if we use `Sport`

In [48]:
train_df[train_df['Genre'] == 'Sport']

,Genre,File_Name,Script
1859,Sport,file_922.txt,SPEED RACER\n\...


In [50]:
train_df = train_df.drop(1859, axis=0)
train_df.shape

(1977, 3)

In [30]:
# !pip install keras
# !pip install nltk

     |████████████████████████████████| 829kB 874kB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /Users/anurag/opt/anaconda3/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/54/j_0l_c55427fvv9sfj94g4sw0000gp/T/pip-install-f0m4pjgo/xgboost/setup.py'"'"'; __file__='"'"'/private/var/folders/54/j_0l_c55427fvv9sfj94g4sw0000gp/T/pip-install-f0m4pjgo/xgboost/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: /private/var/folders/54/j_0l_c55427fvv9sfj94g4sw0000gp/T/pip-install-f0m4pjgo/xgboost/
    Complete output (83 lines):
    ++ pwd
    + oldpath=/private/var/folders/54/j_0l_c55427fvv9sfj94g4sw0000gp/T/pip-install-f0m4pjgo/xgboost
    + cd ./xgboost/
    + mkdir -p build
    + cd build
    + cmake ..
    -- The CXX compiler identification is unknown

# Import the Modeling Libraries 

In [31]:
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


Using TensorFlow backend.
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7

Let's load the datasets

In [35]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anurag/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Define the Scoring Metric

In [38]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    
    """Multi class version of Logarithmic Loss metric.
    
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

### Let's Use LabelEncoder from scikit-learn to convert Genre labels to integers, 0, 1 2

In [51]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train_df.Genre.values)

### Before going further it is important that we split the data into training and validation sets. We can do it using 
#### `train_test_split` from the `model_selection` module of scikit-learn.

In [52]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.Script.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [53]:
print (xtrain.shape)
print (xvalid.shape)

(1779,)
(198,)


# Building Basic Models

### Let's start building our very first model. 

### Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [54]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

## TFIDF on test data

In [59]:
x_test_tfv = tfv.transform(test_df['Script'].values)

In [55]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 2.477 


# Submission 

### Let's predict on the entire test data...

In [92]:
test_set_preds = pd.DataFrame(columns = train_df.Genre.unique().tolist())
test_set_preds.insert(0, 'File_name', test_df.File_Name)

In [93]:
test_set_preds[train_df.Genre.unique().tolist()] = clf.predict_proba(x_test_tfv)

In [94]:
test_set_preds.shape

(849, 22)

In [95]:
test_set_preds.head()

,File_name,Fantasy,Comedy,Drama,Sci-Fi,Romance,Thriller,Adventure,Mystery,Action,...,Horror,Family,Biography,Musical,War,Western,Music,History,Short,Film-Noir
0,file_2300.txt,0.062685,0.0422617,0.0130921,0.00389762,0.127289,0.0614344,0.197159,0.0140578,0.0354991,...,0.0456963,0.00478094,0.00889435,0.0365217,0.0728687,0.0422514,0.00390061,0.202613,0.0102371,0.00661853
1,file_809.txt,0.0618992,0.0426023,0.0129115,0.0038621,0.162966,0.064689,0.196208,0.014118,0.0339208,...,0.0407933,0.00475294,0.00878433,0.0327669,0.156604,0.0394729,0.00381307,0.0953924,0.0099409,0.00640487
2,file_1383.txt,0.0986969,0.145076,0.0133924,0.00401209,0.0762054,0.0481369,0.137012,0.0143404,0.0357723,...,0.0462861,0.00495651,0.00920828,0.0343589,0.0532498,0.152186,0.00401394,0.0966585,0.0110366,0.00692348
3,file_983.txt,0.0819403,0.0498307,0.0136413,0.00403657,0.1036,0.063973,0.164306,0.014735,0.0365077,...,0.0468645,0.00501576,0.00937608,0.0367154,0.0640602,0.0481265,0.00406294,0.226756,0.0109584,0.00690064
4,file_1713.txt,0.1548,0.110987,0.0142456,0.0041241,0.0726372,0.0500778,0.186908,0.0154501,0.037845,...,0.0464402,0.00508516,0.00966895,0.0722427,0.0491728,0.0499662,0.00415086,0.0891547,0.0112245,0.00706685


In [96]:
test_set_preds.to_excel('test_set_preds.xlsx', index=False)